In [1]:
import os
import sys

notebook_path = os.getcwd()
project_path = os.path.abspath(os.path.join(notebook_path, "..", "..", ".."))
print(f"Project path: {project_path}")
# Add the project path to the system path
sys.path.insert(0, project_path)

Project path: c:\Users\dalej\Documents\DragonRegen


In [2]:
from src.MetaFort.SysLogs.DatabaseEngine import DatabaseEngine
from src.DataCreator.SchemaGenerators.SchemaMSSQL import SchemaMSSQL

# create database engine object and connection
driver = "ODBC Driver 17 for SQL Server"
server = 'localhost\\SQLEXPRESS01' 
# server = "Andrew=PC\\SQLEXPRESS"
database = "MetaFort"
conn_str = (
    f"DRIVER={driver};"
    f"SERVER={server};"
    f"DATABASE={database};"
    f"Trusted_Connection=yes;"
)
db_engine = DatabaseEngine(conn_str)
db_engine.connect()
print("Connected to the database.")

Connected to the database.


### Schema Generation Sys Table Setup

In [3]:
# Initialize logging table for dynamic sql executions
s_sql_col = SchemaMSSQL.generate_schema_sql(DatabaseEngine.d_log_tables)
print(s_sql_col)

{'dyn_sql_execution_log': '    [execution_id] INT NOT NULL IDENTITY(1,1),\n    [sql_command] NVARCHAR(MAX) NOT NULL,\n    [sql_params] NVARCHAR(MAX) NOT NULL,\n    [start_time] NVARCHAR(MAX) NOT NULL,\n    [stop_time] NVARCHAR(MAX) NOT NULL,\n    [status] NVARCHAR(MAX) NOT NULL,\n    [user_name] NVARCHAR(MAX) NOT NULL,\n    [process_id] NVARCHAR(MAX) NOT NULL,\n    [process_login_time] DATETIME2 NOT NULL,\n    [error_message] NVARCHAR(MAX) NULL,\n    [error_timestamp] NVARCHAR(MAX) NULL,\n    [metadata] NVARCHAR(MAX) NULL'}


In [4]:
# execute with logging using the database engine
db_engine.create_table('dyn_sql_execution_log', s_sql_col['dyn_sql_execution_log'])
print("Created table for dynamic sql executions")

Created table for dynamic sql executions


# Create the AI Logging Tables

In [5]:
from src.MetaFort.AILoggingTables import AILoggingTables
from pprint import pprint

s_sql_col = SchemaMSSQL.generate_schema_sql(AILoggingTables.d_system_tables)
pprint(s_sql_col)


{'ai_actions_log': '    [action_id] INT NOT NULL IDENTITY(1,1),\n'
                   '    [action_name] NVARCHAR(MAX) NOT NULL,\n'
                   '    [action_version] NVARCHAR(MAX) NOT NULL,\n'
                   '    [parent_action_id] INT NULL,\n'
                   '    [group_action_id] INT NULL,\n'
                   '    [description] NVARCHAR(MAX) NULL,\n'
                   '    [sequence_number] INT NOT NULL,\n'
                   '    [created_dt] DATETIME2 NOT NULL,\n'
                   '    [updated_dt] DATETIME2 NOT NULL,\n'
                   '    [step_name] NVARCHAR(MAX) NOT NULL,\n'
                   '    [step_status] NVARCHAR(MAX) NOT NULL,\n'
                   '    [error_code] NVARCHAR(MAX) NULL,\n'
                   '    [error_message] NVARCHAR(MAX) NULL,\n'
                   '    [error_timestamp] DATETIME2 NULL,\n'
                   '    [metadata] NVARCHAR(MAX) NULL',
 'ai_metrics_log': '    [metric_id] INT NOT NULL IDENTITY(1,1),\n'
              

In [6]:
# execute with logging using the database engine
for table_name, table_schema in s_sql_col.items():
    print(f"===> Creating table {table_name} with schema: \n{table_schema}")
    db_engine.create_table(table_name, table_schema)

===> Creating table ai_actions_log with schema: 
    [action_id] INT NOT NULL IDENTITY(1,1),
    [action_name] NVARCHAR(MAX) NOT NULL,
    [action_version] NVARCHAR(MAX) NOT NULL,
    [parent_action_id] INT NULL,
    [group_action_id] INT NULL,
    [description] NVARCHAR(MAX) NULL,
    [sequence_number] INT NOT NULL,
    [created_dt] DATETIME2 NOT NULL,
    [updated_dt] DATETIME2 NOT NULL,
    [step_name] NVARCHAR(MAX) NOT NULL,
    [step_status] NVARCHAR(MAX) NOT NULL,
    [error_code] NVARCHAR(MAX) NULL,
    [error_message] NVARCHAR(MAX) NULL,
    [error_timestamp] DATETIME2 NULL,
    [metadata] NVARCHAR(MAX) NULL
===> Creating table ai_requests_log with schema: 
    [request_id] INT NOT NULL IDENTITY(1,1),
    [action_id] INT NOT NULL,
    [insert_dt] DATETIME2 NOT NULL,
    [status] NVARCHAR(MAX) NULL,
    [ai_service] NVARCHAR(MAX) NOT NULL,
    [model] NVARCHAR(MAX) NOT NULL,
    [request_parameters] NVARCHAR(MAX) NULL,
    [user_prompt] NVARCHAR(MAX) NULL,
    [engineered_prompt

In [7]:
db_engine.commit()